<a href="https://colab.research.google.com/github/thobranifurqon/FA_BigData/blob/master/PySpark_DataProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalasi Spark pada Google Colab

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.6.tgz
!tar xf spark-2.4.6-bin-hadoop2.6.tgz
!pip install -q findspark

In [ ]:
import os
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.6"

Import library Spark yang sudah diinstal

In [ ]:
import findspark
findspark.init("spark-2.4.6-bin-hadoop2.6")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import col, avg



Inisialisasi SparkContext dan SparkSession


In [ ]:
sc = spark.sparkContext
spark = SparkSession(sc)

Load Data_Set

In [8]:
avocado_df = spark.read.csv("/content/sample_data/avocado.csv", header=True, inferSchema=True)
avocado_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- AveragePrice: double (nullable = true)
 |-- Total Volume: double (nullable = true)
 |-- 4046: double (nullable = true)
 |-- 4225: double (nullable = true)
 |-- 4770: double (nullable = true)
 |-- Total Bags: double (nullable = true)
 |-- Small Bags: double (nullable = true)
 |-- Large Bags: double (nullable = true)
 |-- XLarge Bags: double (nullable = true)
 |-- type: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- region: string (nullable = true)



In [9]:
avocado_df.show(15)

+---+-------------------+------------+------------+-------+---------+------+----------+----------+----------+-----------+------------+----+------+
|_c0|               Date|AveragePrice|Total Volume|   4046|     4225|  4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|region|
+---+-------------------+------------+------------+-------+---------+------+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27 00:00:00|        1.33|    64236.62|1036.74| 54454.85| 48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20 00:00:00|        1.35|    54876.98| 674.28| 44638.81| 58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13 00:00:00|        0.93|   118220.22|  794.7|109149.67| 130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06 00:00:00|        1.08|    78992.15| 1132.0| 71976.41| 72.58|   5811.16|    5677.4|    133.76|        0

Pemrosesan Data

In [12]:
#filter kolom AveragePrice yang nilainya 1 
avocado_AveragePrice = avocado_df.filter(avocado_df.AveragePrice == 1)
#jumlah total averageprice
print("jumlah AveragePrice:", avocado_AveragePrice.count())
#perhitungan rata-rata AveragePrice Avocado berdasarkan year dan region
avocado_AveragePrice.groupby("region").avg("year").show()

jumlah AveragePrice: 167
+-----------------+------------------+
|           region|         avg(year)|
+-----------------+------------------+
|    PhoenixTucson|            2016.0|
|    SouthCarolina|            2015.8|
|          TotalUS|            2015.2|
| WestTexNewMexico|            2016.0|
|       Louisville|            2016.0|
|       Sacramento|2015.3333333333333|
|    DallasFtWorth|            2016.2|
|     Indianapolis|            2017.0|
|        Nashville|            2016.0|
|         LasVegas|            2016.0|
|       GreatLakes|2015.3333333333333|
|          Detroit| 2015.857142857143|
|         Portland|            2016.0|
| CincinnatiDayton|            2017.0|
|         SanDiego|            2017.0|
|            Boise|           2015.75|
|          StLouis|           2016.25|
| NewOrleansMobile|            2015.0|
|         Columbus|            2015.5|
|MiamiFtLauderdale|            2016.0|
+-----------------+------------------+
only showing top 20 rows



Pemrosesan Menggunakan SparkSQL

In [15]:
avocado_df.createOrReplaceTempView("avocado_sql")

result = spark.sql("SELECT * FROM avocado_sql WHERE AveragePrice = 1 LIMIT 15")
result.show()

+---+-------------------+------------+------------+---------+---------+---------+----------+----------+----------+-----------+------------+----+---------+
|_c0|               Date|AveragePrice|Total Volume|     4046|     4225|     4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|   region|
+---+-------------------+------------+------------+---------+---------+---------+----------+----------+----------+-----------+------------+----+---------+
|  5|2015-11-22 00:00:00|         1.0|   356414.57|269799.03|  29732.3|   501.16|  56382.08|   36992.3|  19389.78|        0.0|conventional|2015|  Atlanta|
| 34|2015-05-03 00:00:00|         1.0|    634213.1|538403.48| 28568.26|  1059.73|  66181.63|  32520.82|  33660.81|        0.0|conventional|2015|  Atlanta|
| 40|2015-03-22 00:00:00|         1.0|   479590.62|393591.03|  23419.6|   304.89|   62275.1|  40495.12|  21766.54|      13.44|conventional|2015|  Atlanta|
| 51|2015-01-04 00:00:00|         1.0|   435021.49|364302.39| 23821.16

In [16]:
spark.stop()